<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Как-связаны-параметры-линейной-регрессии-в-исходной-задаче-и-в-преобразованной?" data-toc-modified-id="Как-связаны-параметры-линейной-регрессии-в-исходной-задаче-и-в-преобразованной?-2.0.1"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>Как связаны параметры линейной регрессии в исходной задаче и в преобразованной?</a></span></li></ul></li></ul></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li></ul></div>

# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуем корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

Набор данных находится в файле /datasets/insurance.csv.

- Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
- Целевой признак: количество страховых выплат клиенту за последние 5 лет.

## Загрузка данных

In [2]:
# импортируем необходимые библиотеки

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import numpy as np
from numpy.linalg import inv

In [3]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv')
def info(data):
    print(data.info())
    print()
    print(data.head(10))
    print()
    print(data.duplicated().sum())
info(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None

   Пол  Возраст  Зарплата  Члены семьи  Страховые выплаты
0    1     41.0   49600.0            1                  0
1    0     46.0   38000.0            1                  1
2    0     29.0   21000.0            0                  0
3    0     21.0   41700.0            2                  0
4    1     28.0   26100.0            0                  0
5    1     43.0   41000.0            2                  1
6    1     39.0   39700.0            2                  0
7    1     25.0   38600.0            4                  0
8    1     36.0   49700.0            1                  0
9    1     32.0   51700.0            1         

# Вывод

Пропусков нет. Типы данных числовые - признаки численные. Отношение количества дублией к общему числу клиентов достаточно мало, что бы дубли оставить. Предобработка не требуется. Стоит отметить 2 фактора:
- огромный разрыв между числовыми значениями признаков, негативно скажется на модели без соответствующего масштабирования
- данные в некоторой степени обезличены

## Умножение матриц

Одним из возможных способов защиты персональных данных клинетов, может быть шифрование методом умножения матриц. Проверим возможность реализации и эффективность данного способа. 

Стоит отметить что на шифрованных персональных данных модель (чего угодно, применяемая в бизнесе) должна работать не хуже, чем на не шифрованных. Так же проверим такую возможность

In [4]:
#Определим обучающую `train` и валидационную `valid` выбороки в пропорциях 3:1
#Разделим каждую выборку на `features` -  признаки и `target` — целевой признак 

train, valid = train_test_split(df, test_size=0.25, random_state=12345)

def features_target_split(data, point):
    features = data.drop([point], axis=1)
    target = data[point]
    return features, target

train_features, train_target = features_target_split(train,'Страховые выплаты')
valid_features, valid_target = features_target_split(valid,'Страховые выплаты')

#Масштабируем признаки

def data_to_StandardScaler(data, train_features):
    scaler = StandardScaler()
    scaler.fit(train_features)
    data = scaler.transform(data)
    return data

valid_features_scale = data_to_StandardScaler(valid_features, train_features)
train_features_scale = data_to_StandardScaler(train_features, train_features)

#Обучим модель и посмотрим R2

def model_R2(train_features, train_target, valid_features, valid_target):
    model = LinearRegression().fit(train_features, train_target)
    predicted = model.predict(valid_features)
    r2 = r2_score(valid_target,predicted)
    print('R2 модели {:.2f}'.format(r2))
    return predicted

clean_data  = model_R2(train_features_scale, train_target, valid_features_scale, valid_target)

#Посмотрим R2 нашей собвственной модели на основе матричных вычислений

class LinearRegression_o:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = ((inv(X.T @ X)) @ X.T) @ y
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

def model_R2_o(train_features, train_target, valid_features, valid_target):
    model = LinearRegression_o()
    model.fit(train_features, train_target)
    predicted = model.predict(valid_features)
    r2 = r2_score(valid_target,predicted)
    print('R2 собственной модели {:.2f}'.format(r2))
    return predicted

clean_data_o  = model_R2_o(train_features_scale, train_target, valid_features_scale, valid_target)

R2 модели 0.44
R2 собственной модели 0.44


Значения ошибки моделей работающей на не шифрованных персональных данных зафиксируем
- R2 модели 0.44
- R2 собственной модели 0.44

In [5]:
#Зашифруем Признаки умножив на обратимую матрицу

#Создаем случайную КВАДРАТНУЮ матрицу 
A = np.random.normal(0 , 1, size = (train_features.shape[1],train_features.shape[1]))

#Здесь проверяем обратимость матрицы
check_A= inv(A)

#Шифруем Признаки, масштрабируем их, обучаем модель и смотрим R2
train_features_encode = train_features @ A 
valid_features_encode = valid_features @ A

valid_features_encode_scale = data_to_StandardScaler(valid_features_encode, train_features_encode)
train_features_encode_scale = data_to_StandardScaler(train_features_encode, train_features_encode)

encode_data  = model_R2(train_features_encode_scale, train_target, valid_features_encode_scale, valid_target)
encode_data_o  = model_R2_o(train_features_encode_scale, train_target, valid_features_encode_scale, valid_target)



R2 модели 0.44
R2 собственной модели 0.44


<b>После шифрования Признаков, качество модели не ухудшилось</b>

#### Как связаны параметры линейной регрессии в исходной задаче и в преобразованной?

Предсказания высчитываются по формуле:

$$
a = Xw
$$

Т.к. новая матрица признаков получается умножением исходной $Х$ на случайную обратимую квадратную(обозначим $A$), то формула применит вид:

$$
a = X A w
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Перепишем формулу нахождения весов $w'$:

$$
w' = ((XA)^T XA)^{-1} (XA)^T y
$$

Раскроем скобки:

$$
w' = (XA)^{-1}((XA)^T)^{-1}(XA)^T y
$$

$$
w' = A^{-1} X^{-1} (X^T)^{-1} (A^T)^{-1} A^T X^T y
$$

$$
w' = A^{-1} X^{-1} (X^T)^{-1} (A^{-1})^T A^T X^T y
$$

Используя некоторые свойства матриц сократим запись:

$$
w' = A^{-1}  (X^T X)^{-1} (A A^{-1})^T X^T y
$$


$ (A A^{-1})^T$ это единичная матрица $E$, матрица умножаясь на единичную равна себе, поэтому сократим запись:

$$
w' =  A^{-1}(X^T X)^{-1} X^T y
$$ 

То есть новые веса $w'$ выражаются через исходные $w$ следующим образом:
$$
w' =  A^{-1} w
$$ 


# Вывод

**Ответ:** 
После шифрования Признаков, качество модели не ухудшилось

**Обоснование:** 
Признаки исходной матрицы и преобразованной выражаются через коэффициенты(веса w), поэтому качество линейной регресии не изменилось

## Алгоритм преобразования

In [5]:
#Напишем функцию автоматизации шифрования, она будет принимать на вход выборку и Целевой признак, и:
#разделять ее на обучающую и валидационную,
#выделять в них Признаки и Целевой признак,
#шифровать Признаки,
#возвращать зашифрованые Признаки
#возвращать Целевой признак обучающей и валидационной выборок
#возвращать "ключ шифрования"


def encoder(data, point):
    train, valid = train_test_split(data, test_size=0.25, random_state=12345)

    def features_target_split(data_after_split, point):
        features = data_after_split.drop([point], axis=1)
        target = data_after_split[point]
        return features, target

    train_features, train_target = features_target_split(train , point)
    valid_features, valid_target = features_target_split(valid, point)
    

    A = np.random.normal(0 , 1, size = (train_features.shape[1],train_features.shape[1]))
    check_A= inv(A)
    train_features_encode = train_features @ A 
    valid_features_encode = valid_features @ A
    
    #В выводе добавим не шифрованые train_features , valid_features, что бы посмотреть расшифровку в дальнейшем
    return train_features , valid_features, train_features_encode, valid_features_encode, train_target, valid_target, A
    

#Cмотрим на зашифрованные признаки, видим, что ничего не понятно
clean_f1, clean_f2, encoded_f1, encoded_f2, encoded_t1, encoded_t2, key = encoder(df, 'Страховые выплаты')
print(encoded_f1.head())
print()
print(encoded_f2.head())

                 0             1             2             3
3369  35811.628576  -8269.855050 -20953.159202  34809.321613
1441  56978.063890 -13186.216818 -33385.643385  55404.602517
571   40657.866965  -9403.123765 -23811.195448  39527.827228
225   44615.052480 -10317.568582 -26127.539845  43374.572998
2558  50054.070452 -11581.658262 -29322.498540  48666.093927

                 0            1             2             3
3183  38578.874869 -8920.572284 -22588.289452  37502.202525
1071  42638.516583 -9847.669439 -24947.844908  41442.463733
2640  41647.458341 -9626.506492 -24383.445105  40489.273741
2282  34424.995587 -7967.391981 -20171.229588  33473.322150
1595  39569.362383 -9143.639262 -23158.373297  38460.751289


In [6]:
#Напишем функцию автоматизации расшифрования по "ключу", она будет принимать на вход Зашифрованные Признаки, и:
#Возвращать расшифрованные Признаки

def decoder(data,key):
    decoded_data = abs(round(data.dot(inv(key))))
    return decoded_data

#Проверяем качество расшифровки
decoded_f1 = decoder(encoded_f1, key)
print(decoded_f1)
print()
print(clean_f1)
decoded_f2 = decoder(encoded_f2, key)
print(decoded_f2)
print()
print(clean_f2)

        0     1        2    3
3369  1.0  43.0  36200.0  1.0
1441  1.0  34.0  57600.0  0.0
571   0.0  32.0  41100.0  1.0
225   0.0  36.0  45100.0  1.0
2558  0.0  33.0  50600.0  2.0
...   ...   ...      ...  ...
3497  0.0  42.0  32100.0  0.0
3492  0.0  28.0  22700.0  4.0
2177  1.0  41.0  44700.0  1.0
3557  0.0  22.0  50100.0  4.0
4578  0.0  19.0  40800.0  0.0

[3750 rows x 4 columns]

      Пол  Возраст  Зарплата  Члены семьи
3369    1     43.0   36200.0            1
1441    1     34.0   57600.0            0
571     0     32.0   41100.0            1
225     0     36.0   45100.0            1
2558    0     33.0   50600.0            2
...   ...      ...       ...          ...
3497    0     42.0   32100.0            0
3492    0     28.0   22700.0            4
2177    1     41.0   44700.0            1
3557    0     22.0   50100.0            4
4578    0     19.0   40800.0            0

[3750 rows x 4 columns]
        0     1        2    3
3183  0.0  33.0  39000.0  4.0
1071  0.0  50.0  43100.0 

**Алгоритм**
Данные полностью восстановлены после кодирования

**Обоснование**

Почему качество линейной регрессии не поменяется?
Предсказания высчитываются по формуле:

$$
a = Xw
$$

Т.к. новая матрица признаков получается умножением исходной $Х$ на случайную обратимую квадратную(обозначим $A$), то формула применит вид:

$$
a' = X A w'
$$

Подставив вычисленное значение весов $w' =  A^{-1} w$ получаем:

$$
a' = X A  A^{-1} w
$$

$A A^{-1}$ это еденичная матрица $E$, т.е. можно сократить:

$$
a' = X w = a
$$

Предсказания по исходной и преобразованной матрицам равны.

# Вывод

> Предложенный алгоритм преобразования данных справляется для решения задачи.  
Качество линейной регрессии не меняется, т.к. новые признаки равны $X A$, а новые веса $w' =  A^{-1} w$, поэтому $a' = X w = a$.
Новые признаки выражаются через исходную, умножив на случайную матрицу, соответственно и новые коэффициенты весов. Но так как оба множителя в произведении выражают исходные через определенные коэффициенты, то качество линейной регресии не измениться. 

## Проверка алгоритма

Применив матричные операции, Проверим, что качество линейной регрессии из sklearn не отличается до и после преобразования. Применим метрику R2. (Это уже было сделано ранее, просто убедимся что алгоритм шифрования работает как надо)

In [7]:
#Без шифрования
def wo_code(data, point):
    
    train, valid = train_test_split(data, test_size=0.25, random_state=12345)

    def features_target_split(data, point):
        features = data.drop([point], axis=1)
        target = data[point]
        return features, target

    train_features, train_target = features_target_split(train,'Страховые выплаты')
    valid_features, valid_target = features_target_split(valid,'Страховые выплаты')

    def data_to_StandardScaler(data, train_features):
        scaler = StandardScaler()
        scaler.fit(train_features)
        data = scaler.transform(data)
        return data

    valid_features_scale = data_to_StandardScaler(valid_features, train_features)
    train_features_scale = data_to_StandardScaler(train_features, train_features)

    def model_R2(train_features, train_target, valid_features, valid_target):
        model = LinearRegression().fit(train_features, train_target)
        predicted = model.predict(valid_features)
        r2 = r2_score(valid_target,predicted)
        print('R2 модели без шифования {:.4f}'.format(r2))
    
    encode_data  = model_R2(train_features_scale, train_target, valid_features_scale, valid_target)
    
#С шифрованием
def w_code(data, point):
    def encoder(data, point):
        train, valid = train_test_split(data, test_size=0.25, random_state=12345)

        def features_target_split(data_after_split, point):
            features = data_after_split.drop([point], axis=1)
            target = data_after_split[point]
            return features, target

        train_features, train_target = features_target_split(train , point)
        valid_features, valid_target = features_target_split(valid, point)
    

        A = np.random.normal(0 , 1, size = (train_features.shape[1],train_features.shape[1]))
        check_A= inv(A)
        train_features_encode = train_features @ A 
        valid_features_encode = valid_features @ A
    
        return train_features_encode, valid_features_encode, train_target, valid_target
    
    def data_to_StandardScaler(data, train_features):
        scaler = StandardScaler()
        scaler.fit(train_features)
        data = scaler.transform(data)
        return data
    
    valid_features_encode_scale = data_to_StandardScaler(valid_features_encode, train_features_encode)
    train_features_encode_scale = data_to_StandardScaler(train_features_encode, train_features_encode)
    
    def model_R2(train_features, train_target, valid_features, valid_target):
        model = LinearRegression().fit(train_features, train_target)
        predicted = model.predict(valid_features)
        r2 = r2_score(valid_target,predicted)
        print('R2 модели с шифрованием {:.4f}'.format(r2))
    
    encode_data  = model_R2(train_features_encode_scale, train_target, valid_features_encode_scale, valid_target)
    

wo_code(df,'Страховые выплаты')
w_code(df,'Страховые выплаты')

R2 модели без шифования 0.4352
R2 модели с шифрованием 0.4352


Качество модели совпадает.

# Вывод

> Качество метрики R2 для 2 матриц признаков: до преобразования и после равны, а также равны R2 по написанному класс линейной регресии `LinearRegression_o`, что говорит о ее правильном написании в сравнении с моделью из sklearn.

# Общий вывод

**Проведенные исследования позволили сделать следующие выводы**:  

*- Умножив признаки на обратимую квадратную матрицу, качество линейной регрессии не меняется. Это обусловленно тем, что Веса изменились под новые значения, т.е. признаки исходной матрицы и преобразованной выражаются через коэффициенты(веса w), поэтому качество линейной регресии не изменилось* 

*- Предложенный алгоритм преобразования данных справляется для решения задачи шифрования признаков.*  

*- Зашифрованные данные так же пригодны для предсказаний с помощью линейной регресии*  

*- Алгоритм позволяет дешифровать данные в любой момент*  

*- Случайность реализованная в алгоритме каждый раз меняется при запуске, что делает зашифрованные данные каждый раз новыми. Такой вид исходных данных становиться очень сложно восстановить без кода программы.*  

*- Реализованный алгоритм удовлетворяет поставленной задаче Защиты персональных данных клиентов страховой компании*